In [1]:
from openai import OpenAI
from parser import code_to_file, extract_filename, run_code

In [2]:
client = OpenAI(
base_url = 'http://localhost:11434/v1',
api_key='ollama', # required, but unused
)

In [14]:
query = """Write a python program to check if a given number is Fibonacci number. Input:  8, 34, 41
"""
prompt = query + """
    Format the response in the following format:
    1. plan
    2. filename
    3. Code
    
    Do not show any comments. The code should not have multi line comments. Only single line comments are permissible.
    """

In [15]:
response = client.chat.completions.create(
model="mistral-nemo:12b-instruct-2407-q8_0",
messages=[
    {"role": "system", "content": "You are a helpful coding assistant"},
    {"role": "user", "content": prompt}
  ]
)

In [16]:
raw_response = response.choices[0].message.content
print(raw_response)    

**Plan:**
- Check if the number is a perfect fifth power of 5.
- If it is, return True as all Fibonacci numbers are congruent to 0 or ±1 modulo 5.
- If it's not, check if '5 * n^2 + 4' or '5 * n^2 - 4' (where n is an integer) and calculate the corresponding Fibonacci number. If these calculations are equal to the given number, return True.

**Filename:**
`fibonacci_check.py`

**Code:**

```python
def is_fibonacci(n):
    # Check if n is a perfect fifth power of 5
    if n % 5 == 0 or n % 5 == 4:
        return True

    # Check if n can be expressed as '5 * m^2 + 4' or '5 * m^2 - 4'
    for i in range(1, int(n**0.5) + 1):
        if (5 * i ** 2 + 4 == n) or (5 * i ** 2 - 4 == n):
            return True

    # If none of the above conditions are met, n is not a Fibonacci number
    return False

# Test the function with given numbers
nums = [8, 34, 41]
for num in nums:
    print(f"The number {num} is {'a' if is_fibonacci(num) else 'not a'} Fibonacci number.")
```


In [21]:
response_refine = client.chat.completions.create(
model="mistral-nemo:12b-instruct-2407-q8_0",
messages=[
    {"role": "system", "content": "You are a helpful assistant. There are five conditions. Think carefully and follow the conditions strictly."},
    {"role": "user", "content": """You will analyze the text given, follow the instructions and provide the response. 
    Instructions:
    1. Enclose your response in following xml format tags: <code> and </code>. 
    2. Save the filename and code in dictionary format. keys will be 'filename' and 'code'.  {filename_key: filename_value, code_key: code_value}
    3. Within the dictionary, in the code key if you detect the python code use markdown blocks ```python in the beginning and ``` in the end. 
    4. Use escaped triple quotes (`\"\"\"`) for the value of the key instead of triple quotes.
    5. If you detect multi line comments in triple quotes within the value of the code, remove those comments. Use single line comments
    6. Ensure you have followed the correct openings and closings like <code> </code> tags and { } curly brackets
    
    Eample: 
<code>
{
  "filename": "hello.py",
  "code": ```python
  print("Hello world!")
  ```
}
</code>

Here is the text for you: """ + raw_response}
  ]
)

In [22]:
print(response_refine.choices[0].message.content)

<code>
{
  "filename": "fibonacci_check.py",
  "code": ```python
def is_fibonacci(n):
    # Check if n is a perfect fifth power of 5
    if n % 5 == 0 or n % 5 == 4:
        return True

    # Check if n can be expressed as '5 * m^2 + 4' or '5 * m^2 - 4'
    for i in range(1, int(n**0.5) + 1):
        if (5 * i ** 2 + 4 == n) or (5 * i ** 2 - 4 == n):
            return True

    # If none of the above conditions are met, n is not a Fibonacci number
    return False

# Test the function with given numbers
nums = [8, 34, 41]
for num in nums:
    print(f"The number {num} is {'a' if is_fibonacci(num) else 'not a'} Fibonacci number.")
```
}
</code>


In [23]:
x=response_refine.choices[0].message.content

In [24]:
code_to_file(x)

exit code: 0; success


In [25]:
print(x)

<code>
{
  "filename": "fibonacci_check.py",
  "code": ```python
def is_fibonacci(n):
    # Check if n is a perfect fifth power of 5
    if n % 5 == 0 or n % 5 == 4:
        return True

    # Check if n can be expressed as '5 * m^2 + 4' or '5 * m^2 - 4'
    for i in range(1, int(n**0.5) + 1):
        if (5 * i ** 2 + 4 == n) or (5 * i ** 2 - 4 == n):
            return True

    # If none of the above conditions are met, n is not a Fibonacci number
    return False

# Test the function with given numbers
nums = [8, 34, 41]
for num in nums:
    print(f"The number {num} is {'a' if is_fibonacci(num) else 'not a'} Fibonacci number.")
```
}
</code>


In [26]:
file_name = extract_filename(x)
print(file_name)

fibonacci_check.py


In [27]:
run_code(file_name,'septimus')

'The number 8 is not a Fibonacci number.\nThe number 34 is a Fibonacci number.\nThe number 41 is a Fibonacci number.'